# customer_hm

In [1]:
import pandas as pd
customer = pd.read_csv("h&m_dataset/customer_hm.csv")
customer.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,NONE,25
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,NONE,24
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,NONE,54
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52


In [2]:
customer.info()

<class 'pandas.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   customer_id             1048575 non-null  str  
 1   FN                      1048575 non-null  int64
 2   Active                  1048575 non-null  int64
 3   club_member_status      1048575 non-null  str  
 4   fashion_news_frequency  1048574 non-null  str  
 5   age                     1048575 non-null  int64
dtypes: int64(3), str(3)
memory usage: 48.0 MB


In [3]:
customer.shape

(1048575, 6)

In [6]:
na_c = customer.isna().sum().sort_values(ascending=False) 
na_c.head(3)

fashion_news_frequency    1
customer_id               0
FN                        0
dtype: int64

# 전처리 작업

## fashion_news_frequency 결측치 제거

In [7]:
customer_drop = customer.dropna(subset=['fashion_news_frequency'])

In [9]:
na_c = customer_drop.isna().sum().sort_values(ascending=False) 
na_c

customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
dtype: int64

## age 이상치 제거

In [10]:
customer_drop.describe().round(3)

,FN,Active,age
count,1048574.000,1048574.000,1048574.000
mean,0.356,0.346,36.369
std,0.479,0.476,14.309
min,0.000,0.000,16.000
25%,0.000,0.000,24.000
50%,0.000,0.000,32.000
75%,1.000,1.000,49.000
max,1.000,1.000,99.000


약 36% 정도가 Fashion News 구독  
약 35% 정도가 Fashion News 구독 & Active  
???

In [11]:
# age가 80 이상인 데이터의 건수 합계
count_80 = len(customer_drop[customer_drop['age'] >= 80])

print(f"80세 이상 데이터 건수: {count_80}건")

80세 이상 데이터 건수: 1147건


In [12]:
total_count = len(customer_drop)
percentage = (count_80 / total_count) * 100

print(f"80세 이상 비중: {percentage:.4f}%")

80세 이상 비중: 0.1094%


Age의 Min: 16, Max: 99 <br>
→ 80세 이상 데이터는 전체 데이터의 0.1% 수준이고, 너무 고령이라 분석에서 유의미한 인사이트를 도출하기에는 필요하지 않다고 판단하여 제거

In [13]:
# 고령 고객 제외 (80세 이상 제거)
customer_drop = customer_drop[customer_drop['age'] < 80]

In [14]:
customer_drop.describe().round(3)

,FN,Active,age
count,1047427.000,1047427.000,1047427.000
mean,0.355,0.346,36.318
std,0.479,0.476,14.233
min,0.000,0.000,16.000
25%,0.000,0.000,24.000
50%,0.000,0.000,32.000
75%,1.000,1.000,49.000
max,1.000,1.000,79.000


In [15]:
customer_drop.info()

<class 'pandas.DataFrame'>
Index: 1047427 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   customer_id             1047427 non-null  str  
 1   FN                      1047427 non-null  int64
 2   Active                  1047427 non-null  int64
 3   club_member_status      1047427 non-null  str  
 4   fashion_news_frequency  1047427 non-null  str  
 5   age                     1047427 non-null  int64
dtypes: int64(3), str(3)
memory usage: 55.9 MB


In [16]:
na_c = customer_drop.isna().sum().sort_values(ascending=False) 
na_c.head(3)

customer_id    0
FN             0
Active         0
dtype: int64

### 나이대 구성

In [17]:
# age 연령대 파생 컬럼 생성 (10s ~)
customer_drop["age_group"] = (customer_drop["age"] // 10) * 10
customer_drop["age_group"] = (
    customer_drop["age_group"]
    .astype(int)
    .astype(str) + "s"
)
# 60대 이상 → 60s+
customer_drop.loc[customer_drop["age"] >= 60, "age_group"] = "60s+"

# 결측 확인
customer_drop[["age_group"]].isna().sum()

age_group    0
dtype: int64

In [18]:
print(customer_drop['age_group'].value_counts().sort_index())

age_group
10s      55256
20s     409196
30s     181111
40s     157445
50s     174715
60s+     69704
Name: count, dtype: int64


### 각 컬럼의 고유값 확인

In [19]:
customer_drop['FN'].value_counts()

FN
0    675069
1    372358
Name: count, dtype: int64

In [20]:
customer_drop['Active'].value_counts()

Active
0    684910
1    362517
Name: count, dtype: int64

In [21]:
customer_drop['club_member_status'].value_counts()

club_member_status
ACTIVE        981597
PRE-CREATE     65471
LEFT CLUB        359
Name: count, dtype: int64

In [22]:
customer_drop['fashion_news_frequency'].value_counts()

fashion_news_frequency
NONE         674002
Regularly    372767
Monthly         658
Name: count, dtype: int64

In [23]:
# 컬럼 고유값 형식 동일하게 변경(ex. ACTIVE → Active)

In [24]:
new_customer = customer_drop.copy()

In [25]:
new_customer['club_member_status'] = customer_drop['club_member_status'].str.title()
new_customer['club_member_status'].value_counts()

club_member_status
Active        981597
Pre-Create     65471
Left Club        359
Name: count, dtype: int64

In [26]:
new_customer['fashion_news_frequency'] = customer_drop['fashion_news_frequency'].str.capitalize()
new_customer['fashion_news_frequency'].value_counts()

fashion_news_frequency
None         674002
Regularly    372767
Monthly         658
Name: count, dtype: int64

### FN 값과 fashion_news_frequency 간 관계 파악